# Импорт библиотек

In [10]:
# Copyright Simonyan Suren
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import statsmodels.api as sm


# Загрузка данных

In [11]:
data = pd.read_csv("/content/fifa_players.csv")

# Информация о данных
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17954 entries, 0 to 17953
Data columns (total 51 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   name                           17954 non-null  object 
 1   full_name                      17954 non-null  object 
 2   birth_date                     17954 non-null  object 
 3   age                            17954 non-null  int64  
 4   height_cm                      17954 non-null  float64
 5   weight_kgs                     17954 non-null  float64
 6   positions                      17954 non-null  object 
 7   nationality                    17954 non-null  object 
 8   overall_rating                 17954 non-null  int64  
 9   potential                      17954 non-null  int64  
 10  value_euro                     17699 non-null  float64
 11  wage_euro                      17708 non-null  float64
 12  preferred_foot                 17954 non-null 

# Удаление ненужных данных

In [12]:
# Удаление столбцов с отсутствующими значениями и неинформативных столбцов
data = data.drop(columns = ["release_clause_euro", "national_team", "national_rating", "national_team_position",
                           "national_jersey_number", "body_type", "international_reputation(1-5)", "weak_foot(1-5)",
                           "skill_moves(1-5)", "potential", "full_name", "birth_date"], axis = 1)

# Удаление строк с отсутствующими значениями
data = data.dropna()

# Исключение вратарей
data = data.query("positions != 'GK'")

# Проверка первых строк после очистки
data.head()

,name,age,height_cm,weight_kgs,positions,nationality,overall_rating,value_euro,wage_euro,preferred_foot,...,long_shots,aggression,interceptions,positioning,vision,penalties,composure,marking,standing_tackle,sliding_tackle
0,L. Messi,31,170.18,72.1,"CF,RW,ST",Argentina,94,110500000.0,565000.0,Left,...,94,48,22,94,94,75,96,33,28,26
1,C. Eriksen,27,154.94,76.2,"CAM,RM,CM",Denmark,88,69500000.0,205000.0,Right,...,89,46,56,84,91,67,88,59,57,22
2,P. Pogba,25,190.50,83.9,"CM,CAM",France,88,73000000.0,255000.0,Right,...,82,78,64,82,88,82,87,63,67,67
3,L. Insigne,27,162.56,59.0,"LW,ST",Italy,88,62000000.0,165000.0,Right,...,84,34,26,83,87,61,83,51,24,22
4,K. Koulibaly,27,187.96,88.9,CB,Senegal,88,60000000.0,135000.0,Right,...,15,87,88,24,49,33,80,91,88,87



# Создание новых признаков

In [13]:
# Интегральные показатели навыков
data['physical'] = (
    data["strength"] + data["sprint_speed"] + data["agility"] + data["reactions"] +
    data["stamina"] + data["jumping"] + data["balance"] + data["acceleration"]
) / 8
data['defensive_skill'] = (
    data["sliding_tackle"] + data["standing_tackle"] + data["interceptions"] +
    data["marking"] + data["positioning"] + data["aggression"]
) / 6
data["football_skills"] = (
    data["ball_control"] + data["short_passing"] + data["long_passing"] + data["composure"] +
    data["vision"] + data["dribbling"]
) / 6
data["offensive_skills"] = (
    data["crossing"] + data["finishing"] + data["long_shots"] + data["volleys"] +
    data["heading_accuracy"]
) / 5

# Преобразование стоимости игроков в логарифмическую шкалу
data["value_euro"] = np.log(data["value_euro"])

# Создание признака для доминирующей ноги
left_foot = [1 if x == "Left" else 0 for x in data["preferred_foot"]]
data["left"] = left_foot

# Отбор признаков для кластеризации
X = data[[
    'offensive_skills', 'football_skills', 'defensive_skill', 'physical',
    'age', 'height_cm', 'weight_kgs', 'freekick_accuracy', 'penalties', 'left'
]]

# Стандартизация данных
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Задача кластеризации

In [14]:
# Метод KMeans
kmeans = KMeans(n_clusters=5, random_state=42)
kmeans_labels = kmeans.fit_predict(X_scaled)
kmeans_silhouette = silhouette_score(X_scaled, kmeans_labels)

# Метод DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=10)
dbscan_labels = dbscan.fit_predict(X_scaled)
dbscan_silhouette = silhouette_score(X_scaled, dbscan_labels) if len(set(dbscan_labels)) > 1 else -1

# Агломеративная кластеризация
agglo = AgglomerativeClustering(n_clusters=5)
agglo_labels = agglo.fit_predict(X_scaled)
agglo_silhouette = silhouette_score(X_scaled, agglo_labels)


# Вывод результатов

In [17]:
kmeans = KMeans(n_clusters=5, random_state=42)
kmeans_labels = kmeans.fit_predict(X_scaled)
kmeans_silhouette = silhouette_score(X_scaled, kmeans_labels)
print(f"Число кластеров KMeans: 5")
print(f"Средний силуэтный коэффициент KMeans: {kmeans_silhouette}")

dbscan = DBSCAN(eps=1.5, min_samples=10)
dbscan_labels = dbscan.fit_predict(X_scaled)
dbscan_clusters = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)
dbscan_silhouette = silhouette_score(X_scaled, dbscan_labels) if dbscan_clusters > 1 else -1
print(f"Число кластеров DBSCAN (включая шум): {dbscan_clusters}")
print(f"Средний силуэтный коэффициент DBSCAN: {dbscan_silhouette}")

agglo = AgglomerativeClustering(n_clusters=5)
agglo_labels = agglo.fit_predict(X_scaled)
agglo_silhouette = silhouette_score(X_scaled, agglo_labels)
print(f"Число кластеров Агломеративной кластеризации: 5")
print(f"Средний силуэтный коэффициент Агломеративной кластеризации: {agglo_silhouette}")


Число кластеров KMeans: 5
Средний силуэтный коэффициент KMeans: 0.14878774473957262
Число кластеров DBSCAN (включая шум): 2
Средний силуэтный коэффициент DBSCAN: 0.133951626318541
Число кластеров Агломеративной кластеризации: 5
Средний силуэтный коэффициент Агломеративной кластеризации: 0.11399375022812255


# Общий вывод

**KMeans**: Метод с 5 кластерами показал средний силуэтный коэффициент 0.1488, что свидетельствует о слабой, но все же существующей структуре данных. Это указывает на то, что кластеризация по выбранным признакам не идеальна, и требуется дополнительная настройка или пересмотр признаков.

**DBSCAN**: Этот метод выявил только один кластер с шумом, что означает, что DBSCAN не смог разделить данные на несколько четких групп. Средний силуэтный коэффициент был равен -1, что указывает на полное отсутствие кластеризации. Это может быть связано с неподходящими параметрами, такими как слишком большой радиус (eps) или слишком маленькое количество минимальных точек для формирования кластеров.

**Агломеративная кластеризация**: С 5 кластерами, агломеративный метод дал силуэтный коэффициент 0.1140, что тоже подтверждает наличие слабых, но все же разделимых кластеров. Однако результат оказался немного хуже по сравнению с KMeans.

KMeans и Агломеративная кластеризация показали схожие результаты, но их силуэтный коэффициент указывает на не очень качественную кластеризацию. Стоит пересмотреть выбор признаков и, возможно, применить другие методы предобработки данных или масштабирования.
Для DBSCAN следует провести настройку параметров (например, уменьшить радиус или увеличить минимальное количество точек в кластере), чтобы улучшить результаты кластеризации.
Для повышения качества кластеризации можно попробовать использовать другие признаки или методы анализа данных, а также протестировать различные алгоритмы и их настройки.